<a href="https://colab.research.google.com/github/AbeerProg/RRDS/blob/main/FSPronoun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [28]:
import pandas as pd
import spacy
import string

# Load SpaCy English model
nlp = spacy.load('en_core_web_sm')

# Load the dataset from the specified Excel file
df = pd.read_excel('1.xlsx')

# Check if the column name 'Text' exists
if 'text' not in df.columns:
    print("Error: The DataFrame does not contain a 'Text' column.")
else:
    # Set of first-person singular pronouns to preserve
    first_person_singular_pronouns = {"i", "me", "my", "mine", "myself"}

    def preprocess_text(text):
        # Process the text with SpaCy
        doc = nlp(text.lower())  # Lowercase the text

        # Retain words, skip punctuation, and apply selective lemmatization
        processed_tokens = []
        for token in doc:
            if token.is_punct:
                continue
            if token.text in first_person_singular_pronouns:
                processed_tokens.append(token.text)  # Preserve the pronoun as is
            elif not token.is_stop:
                processed_tokens.append(token.lemma_)  # Lemmatize non-stop words

        # Join tokens back into a single string
        processed_text = ' '.join(processed_tokens)
        return processed_text

    def calculate_pronoun_ratio(text):
        if not text:
            return 0.0

        # Process the preprocessed text
        doc = nlp(text)

        total_words = len(doc)
        first_person_singular_count = sum(1 for token in doc if token.text.lower() in first_person_singular_pronouns)

        # Calculate the ratio
        ratio = first_person_singular_count / total_words if total_words > 0 else 0.0
        return ratio

    # Apply the preprocessing function to each review in the DataFrame
    df['preprocessed_text'] = df['text'].apply(preprocess_text)

    # Apply the pronoun ratio calculation to the preprocessed text
    df['pronoun_ratio'] = df['preprocessed_text'].apply(calculate_pronoun_ratio)

    # Display the DataFrame with the new columns
    print(df[['text', 'preprocessed_text', 'pronoun_ratio']])

    # Save the updated DataFrame to a new Excel file
    df.to_excel('processed_output.xlsx', index=False)


                                                    text  \
0                                    Need more munchkins   
1                                         Nice as always   
2                                     Great little place   
3                               Service was exceptional!   
4                           Perfect breakfast everytime.   
...                                                  ...   
25115  Flavors were all pretty good, very fresh clean...   
25116  I usually do not make reviews unless I think i...   
25117  TLDR in terms of Covid-19 safety and general d...   
25118  I've ordered from this restaurant twice, and i...   
25119  I have been working my way through the menu si...   

                                       preprocessed_text  pronoun_ratio  
0                                          need munchkin       0.000000  
1                                                   nice       0.000000  
2                                     great little place 